In [10]:
import requests, time, csv, os
import random
from datetime import datetime, timedelta
import cloudinary
import cloudinary.uploader

cloudinary.config(
  cloud_name = "dvxmaiofh", 
  api_key = "834668983718514", 
  api_secret = "KI_mVAdKhFNvtbJN4w9TWSzJqno",
  secure = True
)

CATEGORY_ID = 8407
MAX_PRODUCTS = 3

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

items = []
page = 1

def random_date(start_year=2022, end_year=2025):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    return (start + timedelta(
        days=random.randint(0, (end - start).days)
    )).strftime("%Y-%m-%d")

FAKE_MATERIALS = [
    "Nhựa ABS",
    "Hợp kim",
    "Thép không gỉ",
    "Silicone",
    "Vải tổng hợp"
]

FAKE_SIZES = ["S", "M", "L"]

FAKE_COLORS = [
    "Đỏ",
    "Xanh dương",
    "Xanh lá",
    "Vàng",
    "Cam",
    "Hồng",
    "Tím",
    "Đen",
    "Trắng",
    "Xám",
    "Nâu"
]

print(f"Collecting products from Tiki category ID: {CATEGORY_ID}")

while len(items) < MAX_PRODUCTS:
    LIST_API = f"https://tiki.vn/api/v2/products?category={CATEGORY_ID}&page={page}&limit=40"
    resp = requests.get(LIST_API, headers=headers).json()
    data = resp.get("data", [])

    if not data:
        break

    for p in data:
        if len(items) >= MAX_PRODUCTS:
            break

        pid = p["id"]
        detail = requests.get(
            f"https://tiki.vn/api/v2/products/{pid}",
            headers=headers
        ).json()

        # ========== BASIC ==========
        product_name = detail.get("name")
        price = detail.get("price")
        stock = random.randint(50, 200)
        type_ = "accessory"

        # ========== IMAGES ==========
        imgList = [
            img["base_url"]
            for img in detail.get("images", [])
            if "base_url" in img
        ][:5]

        if len(imgList) < 1:
            continue

        manufacture_date = random_date(2022, 2024)
        entry_date = random_date(2024, 2025)
        expiry_date = random_date(2026, 2028)

        material = random.choice(FAKE_MATERIALS)
        size = random.choice(FAKE_SIZES)
        color = random.choice(FAKE_COLORS)

        cloudinary_urls = []
        print(f"   >>> Đang upload {len(imgList)} ảnh lên Cloudinary cho SP: {pid}...")
        
        for img_url in imgList:
            if img_url:
                try:
                    upload_result = cloudinary.uploader.upload(img_url, folder="PuppyLobby")
                    cloudinary_urls.append(upload_result['secure_url'])
                except Exception as e:
                    print(f"      [Lỗi Upload] Không thể up ảnh {img_url}: {e}")
                    cloudinary_urls.append(img_url)

        items.append({
            "product_name": product_name,
            "price": price,
            "manufacture_date": manufacture_date,
            "entry_date": entry_date,
            "expiry_date": expiry_date,
            "stock": stock,
            "type": type_,
            "imgList": cloudinary_urls,
            "size": size,
            "color": color,
            "material": material,
        })

        print(f"GET {len(items)} | {product_name[:40]}")
        time.sleep(0.25)

    page += 1
    time.sleep(0.5)

def sql_escape(value):
    if value is None:
        return "NULL"
    return "'" + str(value).replace("'", "''") + "'"

with open("insert_accessory.sql", "w", encoding="utf-8") as f:
    for item in items:
        f.write("-- ================================\n")
        f.write(f"-- {item['product_name']}\n")

        # ===== WITH new_product =====
        f.write("WITH new_product AS (\n")
        f.write("  INSERT INTO product (\n")
        f.write("    product_name, price, manufacture_date, entry_date, expiry_date, stock, type\n")
        f.write("  ) VALUES (\n")
        f.write(f"    {sql_escape(item['product_name'])},\n")
        f.write(f"    {item['price']},\n")
        f.write(f"    {sql_escape(item['manufacture_date'])},\n")
        f.write(f"    {sql_escape(item['entry_date'])},\n")
        f.write(f"    {sql_escape(item['expiry_date'])},\n")
        f.write(f"    {item['stock']},\n")
        f.write(f"    {sql_escape(item['type'])}\n")
        f.write("  ) RETURNING product_id\n")
        f.write(")")

        # ===== productimage CTEs =====
        for idx, img in enumerate(item["imgList"]):
            f.write(",\n")
            f.write(f"img_{idx} AS (\n")
            f.write(
                "  INSERT INTO productimage (product_id, image_url)\n"
                f"  SELECT product_id, {sql_escape(img)} FROM new_product\n"
                ")\n"
            )

        # ===== accessory (FINAL STATEMENT) =====
        f.write(
            "INSERT INTO accessory (product_id, size, color, material)\n"
            "SELECT product_id, "
            f"{sql_escape(item['size'])}, "
            f"{sql_escape(item['color'])}, "
            f"{sql_escape(item['material'])} "
            "FROM new_product;\n\n"
        )

   >>> Đang upload 5 ảnh lên Cloudinary cho SP: 277853374...
GET 1 | Đồ chơi nhựa màu sắc có kèn bíp bíp cho 
   >>> Đang upload 4 ảnh lên Cloudinary cho SP: 276282161...
GET 2 | Đồ Chơi Bóng Dây Thừng Cầm Tay Hình Quả 
   >>> Đang upload 5 ảnh lên Cloudinary cho SP: 276282150...
GET 3 | Bóng Đồ Chơi Bóng Cao Su Cho Chó Hình Qu
